# Get ROI Time Series

In [1]:
def roi_maskdims(img_shape, nROIs):
    if isinstance(img_shape, tuple):
        img_shape = list(img_shape)

    mask_shape = list(img_shape)
    mask_shape.append(nROIs)

    del(img_shape, nROIs)
    return(mask_shape)

In [2]:
def maskROIs_as_time(mat):
    import numpy as np
    import nibabel as nib

    if isinstance(mat, str):
        mat = nib.load(mat)
        mat = img.get_fdata()

    roi_ls = np.unique(mat)
    roi_ls = roi_ls[roi_ls != 0]

    mat_dim = list(mat.shape)
    mat_dim.append(1)
    mat = np.reshape(mat, mat_dim)

    roi_mat = np.tile(mat, (1,1,1,len(roi_ls)))

    for ind,val in zip(range(len(roi_ls)), roi_ls):
        roi_mat[:,:,:,ind] = roi_mat[:,:,:,ind] - (val-1)
    
    roi_mat[roi_mat != 1] = 0

    del(mat, roi_ls, ind, val)
    return(roi_mat)


In [3]:
def mult_NiiByMask(img, mask):
    # for each point along the fourth dimension of mat4d, element-wise multiply mat4d and mat3d
    import numpy as np

    if len(img.shape) != 4:
        raise Exception("img must be a 4-dimensional array")
    if len(mask.shape) != 3:
        raise Exception("mask must be a 3-dimensional array")

    mask_4d_dim = list(mask.shape)
    mask_4d_dim.append(1)
    
    mask = np.reshape(mask, mask_4d_dim)
    mask = np.tile(mask, (1,1,1,img.shape[3]))
    
    prod_mat = np.zeros(img.shape)
    prod_mat[np.nonzero(mask)] = img[np.nonzero(mask)]
    
    del(img, mask)
    return(prod_mat) # output is x,y,z,time


In [4]:
def roi_tcourse(img, mask):
    # compute the mean time course for each ROI 
    import numpy as np

    if img.shape[0:3] != mask.shape:
        raise Exception("img and mask are not in the same 3d space.")

    mean_tcourse = img / np.sum(mask)
    mean_tcourse = np.sum(mean_tcourse, (0,1,2))

    del(img, mask)
    return(mean_tcourse)

In [5]:
def atlas_tcourse(nifti_path, mask_path, file_out=None):
    import numpy as np
    import nibabel as nib

    print("+ IMPORTING DATA")
    img = nib.load(nifti_path)
    img = img.get_fdata()
    
    mask = nib.load(mask_path)
    mask = mask.get_fdata()

    print("+ SPLITTING ROIs")
    mask = maskROIs_as_time(mask) # mask becomes 4d

    if len(img.shape) != 4:
        raise Exception("The provided nifti file must be 4D.")
    if img.shape[0:3] != mask.shape[0:3]:
        raise Exception("The provided nifti file and mask must be in the same 3D space.")
    
    print("+ COMPUTING MEAN TIME COURSE")
    mean_tcourse = np.zeros([img.shape[3], mask.shape[3]])
    
    for roi in range(mask.shape[3]):
        prod_mat = mult_NiiByMask(img, mask[:,:,:,roi]) # output is x,y,z,time
        mean_tcourse[:,roi] = roi_tcourse(prod_mat, mask[:,:,:,roi])
        
        del(prod_mat)

    print("+ SAVING")
    if isinstance(file_out,str):
        hdr = ",".join(["roi_"+str(roi+1) for roi in range(mask.shape[3])])
        np.savetxt(file_out, mean_tcourse, header=hdr, comments='', delimiter=',')

    return(mean_tcourse)

# Sliding Time Window

In [ ]:
def n_twin(n_tpts, win_sz, step_sz, fill_tline=False):
    from math import floor
    
    num_twin = (n_tpts - win_sz) / step_sz
    
    if fill_tline:
        ovr_spill = num_twin.is_integer()
    else:
        ovr_spill = False

    return(floor(num_twin), ovr_spill) # returns the number of whole time windows and if there is spill-over


In [ ]:
def sliding_twin(ttab, win_sz, step_sz, prefix=None, plot=True, fill_tline=False):
    import numpy as np

    if len(ttwin.shape) != 2:
        raise Exception("The table must be a 2-D array")

    num_twin, ovr_spill = n_twin(ttab.shape[1], win_sz, step_sz, fill_tline)

    fc_mats = np.zero([ttab.shape[1], ttab.shape[1], num_twin+ovr_spill])

    win_pos = 0
    for win in range(num_twin):
        ttab_win = ttab[range(win_pos, win_pos+step_sz),:]
        fc_mats[:,:,win] = np.corrcoef(np.transpose(ttab_win))
        win_pos += step_sz

    if ovr_spill:
        ttab_win = ttab[win_pos:-1,:]
        fc_mats[:,:,num_twin+ovr_spill] = np.corrcoef(np.transpose(ttab_win))

    if plot:
        print("plotting") ###

    if prefix is not None:
        import scipy.io as sio
        sio.savemat(prefix+"_FCmatrix.mat", fc_mats)

    return(ttab_win)

# K-Means Clustering

# Testing Area

In [8]:
# mask_path  = "/mnt/c/Users/John/Desktop/MNI/MNI-maxprob-thr0-2mm.nii.gz"
# nifti_path = "/mnt/d/Downloads/ds000031-download/sub-01/ses-003/out/func/scrub_motreg_s_nl_m_t_func.nii.gz"
# file_out   = "/mnt/c/Users/John/Desktop/mean_roi_tcourse.csv"

mask_path  = "C:/Users/John/Desktop/MNI/MNI-maxprob-thr0-2mm.nii.gz"
nifti_path = "D:/Downloads/ds000031-download/sub-01/ses-003/out/func/scrub_motreg_s_nl_m_t_func.nii.gz"
file_out   = "C:/Users/John/Desktop/mean_roi_tcourse.csv"

tcourse = atlas_tcourse(nifti_path, mask_path, file_out)

+ IMPORTING DATA
+ SPLITTING ROIs
+ COMPUTING MEAN TIME COURSE
0
1
2
3
4
5
6
7
8
+ SAVING


In [17]:
import numpy as np

cormat=np.corrcoef(np.transpose(tcourse))
cormat[7,:]

array([0.54641922, 0.87566213, 0.78299617, 0.82603912, 0.96633048,
       0.89760251, 0.70064896, 1.        , 0.78863196])